In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json

# Scrapping

In [45]:
prev_len = 0
links = set()
page = 1

headers={'x-csrf-token': 'eRRty4ktUuh5BcXGSEeeYyDodk7VtVsooysxw3BB',
        'cookie':'XSRF-TOKEN=eRRty4ktUuh5BcXGSEeeYyDodk7VtVsooysxw3BB; trad_session=OzfSlLuDow6StpuAf52gfnAuSOWTh9GvAwmhYKKh; e-customer-key=TM5f08b13ad32a75.89112397;',
        'accept':'application/json',
        'x-requested-with': 'XMLHttpRequest'}
    
sofas = []
total_pages = 9999
while page<=total_pages:
    search = requests.post(f"https://www.trademax.no/m%C3%B8bler/sofaer?page={page}&sorting=popularity", headers=headers,json={'filter':{}})
    resp = search.json()
    if page == 1:
        total_items = resp['pagination']['data']['items']['total']
        total_pages = resp['pagination']['data']['pages']['count']
    sofas += resp['products']['data']
    page+=1
    

json.dump(sofas, open('./data/trademax_items.json', 'w'))
len(sofas)

In [1]:
import json
sofas = json.load(open('./data/trademax_items.json'))
len(sofas)

9525

In [11]:
from datetime import datetime

trans = {'Bredde': 'width',
         'Maks bredde' : 'max_width',         
             'Høyde': 'height',
             'Totalhøyde': 'height',
             'Lengde': 'length',
             'Dybde': 'depth',
             'Maks. dybde':'max_depth',
             'Dypeste del':'max_depth',
             'Min. dybde':'min_depth',
             'Sittehøyde': 'sit_height',
             'Setehøyde': 'sit_height',
             'Sittedybde' : 'sit_depth',
             'Setedybde' : 'sit_depth',
             'Totaldybde sete': 'sit_depth',
             'Maks setedybde': "max_sit_depth",
             'Min. setedybde': "min_sit_depth",
             'Sete bredde' : 'sit_width',
             'Sittebredde' : 'sit_width',
             'Sengebredde': 'bed_width',
             'Sengelengde': 'bed_length',
             'Armlenebredde' : 'arm_width',
             'Armlene bredde' : 'arm_width',
             'Høyde armlene' : 'arm_height',
             'Fri høyde under møbel': 'clearance',
             'Fri høyde under møbler': 'clearance',
             'Benhøyde': 'clearance',
             'Sokkel/Ben høyde': 'clearance',
             'Ben' : 'legs'
            }

print_index = 0

def scrap_trademax(sofa):
    url = 'https://www.trademax.no' + sofa['uri']
    content = requests.get(url).content
    soup = BeautifulSoup(content, 'html.parser')

    def extract(spec_item):
        spec_item['title']
        props = spec_item['properties']
        for p in props:
            name = trans.get(p['name'], p['name'])
            yield name, p['values'][0]['value']

    h = "\n\n        var componentsData = "
    t = next(t.text[len(h):] for t in soup.find_all('script') if t.text.startswith(h))
    oobj = json.loads(t[:-7])
    spec_items = oobj['productInfoTabs']['data']['ProductSpecificationComponent']['specificationItems']['product']

    from itertools import chain
    flat = chain.from_iterable(extract(item) for item in spec_items)
    spec_dict = dict(flat)    
    
    content = requests.get(url).content
    soup = BeautifulSoup(content, 'html.parser')
    
    item = {
        'source' : 'trademax',
        'image' : "https://images.trademax.no/" + sofa['images'][0]['path']+"?hash="+sofa['images'][0]['hash'],
        'description' : oobj['productInfoTabs']['data']['ProductHighlightsComponent']['description'],
        'url' : soup.find('meta', attrs={'property':'og:url'}).attrs['content'],
        'price' : sofa['prices']['current'], #price_amount+' '+price_currency,
        'brand' : sofa['brandName'] #soup.find('meta', attrs={'itemprop':'brand'}).attrs['content']
    }
    
    item.update(sofa)
    
    
    #del (zz['variants'])
    item.pop('analyticsData', None)
    item.pop('uri', None)
    item.pop('brandName', None)
    item.pop('images', None)
    item.pop('prices', None)
    item.pop('labels', None)
    item.pop('discount', None)
    item.pop('customClasses', None)
    item.pop('isAlwaysLowPrice', None)
    item.pop('campaign', None)
    
    global print_index
    item.update(spec_dict)
    if print_index % 200==0:
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time} step:{print_index}")
    print_index += 1
    
    return item

items = (scrap_trademax(sofa) for sofa in sofas)

In [ ]:
trademax_pd = pd.DataFrame(items)
trademax_pd.to_csv('./data/trademax.csv')

# Start here

In [2]:
trademax_pd = pd.read_csv('https://raw.githubusercontent.com/veonua/norske_sofaer/master/data/trademax.csv', low_memory=False)
len(trademax_pd)

9525

In [3]:
import math
def width(row):
    if isinstance(row,str):
        return float(row.split('cm')[0].strip())
    else: return row

def max_depth(row):
    row = row.fillna(0)
    val = row['max_depth'] or row['depth']
    if not val:
        return 0
    return float(val.split('cm')[0].strip())

trademax_pd['width_i'] = trademax_pd['width'].apply(width)
trademax_pd['max_depth_i'] = trademax_pd.apply(max_depth, axis=1)

fil = trademax_pd.drop(columns=['variants', 'description', 'rating', 'isFabric', 'subtitle', 'parent_id','variantsCount', 'id'])


filt = trademax_pd[['title','image', 'Serie', 'width_i', 'max_depth_i', 'clearance', 'url']]
fw = trademax_pd['width_i'].between(260, 280) #| (ikea_pd['width_a'] == 0)
fd = trademax_pd['max_depth_i'].between(175,223) | (trademax_pd['max_depth_i'] == 0)
leather = trademax_pd['title'].str.contains('Lær')
filt = filt[fd & fw & ~ leather].sort_values('Serie')

In [4]:
from IPython.display import HTML
def image_formatter(im):
    return f'<img src="{im}" width="350px">'
def a_formatter(url):
    return f'<a href="{url}">link</a>'

disp = filt
HTML(disp.to_html(formatters={'image': image_formatter, 'url': a_formatter}, escape=False))

,title,image,Serie,width_i,max_depth_i,clearance,url
33,Aldin Hjørnesofa Vendbar,,Aldin,268.0,0.0,5 cm,link
1495,Angola Sovesofa Høyre,,Angola,275.0,0.0,14 cm,link
1202,Bosnia Hjørnesofa,,Bosnia,270.0,175.0,20 cm,link
1109,Bosnia Hjørnesofa,,Bosnia,270.0,175.0,20 cm,link
1036,Bosnia Hjørnesofa,,Bosnia,270.0,175.0,20 cm,link
1800,Bosnia Hjørnesofa,,Bosnia,270.0,175.0,20 cm,link
2129,Boston U-sofa med Divan Høyre,,Boston,276.0,201.0,5 cm,link
2208,Boston U-sofa med Divan Venstre,,Boston,276.0,201.0,5 cm,link
608,Chesterfield Hjørnesofa 280 cm,,Chesterfield,280.0,0.0,13,link
620,Chesterfield Hjørnesofa 280 cm,,Chesterfield,280.0,0.0,13,link
